<a href="https://colab.research.google.com/github/minson18/ML_final_project/blob/main/109550058_Final_inferencce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

以下這段 pip install 執行完要按下輸出區的 RESTART RUNTIME，再繼續往下執行。

In [ ]:
pip install imbalanced-ensemble

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os

# Root Path
os.chdir('/content/drive/MyDrive/ML/final project')

TEST_PATH = "test.csv"
MODEL_PATH = "models"

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
#from sklearn.svm import SVC, SVR
from sklearn.metrics import accuracy_score
import math
import csv
#from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
#from imbalanced_ensemble.ensemble import OverBoostClassifier
#from imbalanced_ensemble.ensemble import SelfPacedEnsembleClassifier
#import kaggle
#from xgboost import XGBClassifier

In [ ]:
# Pre-proccessing Testing data
test_df = pd.read_csv(TEST_PATH)

# translate attribute_0, attribute_1 from string to int
test_df['attribute_0'] = test_df['attribute_0'].replace(
  ['material_7', 'material_5'], [7, 5])
test_df['attribute_1'] = test_df['attribute_1'].replace(
  ['material_7', 'material_5', 'material_6'],[7, 5, 6])

# add column measurement_5_wasmissing, measurement_4_wasmissing
test_df.insert(1, 'measurement_5_wasmissing', 0)
test_df.loc[pd.isna(test_df['measurement_5']),'measurement_5_wasmissing'] = 1
test_df.insert(1, 'measurement_4_wasmissing', 0)
test_df.loc[pd.isna(test_df['measurement_4']),'measurement_4_wasmissing'] = 1

# drop redundant columns
test_df = test_df.drop(['product_code'], axis=1)
# fill Nan
test_df = test_df.fillna(0)

# first dataset with selected features
test_df_select = test_df[[
  "loading", "measurement_17", "measurement_6", "measurement_4", "measurement_2", 
  "measurement_5_wasmissing", "attribute_3", "attribute_1", "measurement_9", 
  "measurement_4_wasmissing", "measurement_7", "attribute_0"
  ]]
x_test_select = test_df_select.to_numpy()

# second dataset with all features
testData = test_df.to_numpy()
x_test = testData[:, 1:]

test_id = testData[:, 0].astype(int)

In [ ]:
from imbalanced_ensemble.ensemble import RUSBoostClassifier
import pickle

def test(x_test, x_test_select):

  # List models names with two different dataset
  models_select = ["model_select_A", "model_select_B"]
  models = ["model_A", "model_B"]

  # Predict probability using each model
  predictions = [] # all predict results
  for model_name in models_select:
    with open(MODEL_PATH + "/" + model_name + '.pickle', 'rb') as f:
      model = pickle.load(f)

    predict = model.predict_proba(x_test_select)
    predict = predict[:, 1]
    predictions.append(predict)
  
  for model_name in models:
    with open(model_name + '.pickle', 'rb') as f:
      model = pickle.load(f)
      
    predict = model.predict_proba(x_test)
    predict = predict[:, 1]
    predictions.append(predict)

  # Get the average of 4 models' prediction
  y_pred = sum(predictions) / len(predictions)
  
  # Concatenate id and failure into submit
  submit = np.array([test_id, y_pred]).T
  submit = submit.tolist()
  # Check each datatype in submit
  for sub in submit:
    sub[0] = int(sub[0])
    sub[1] = float(sub[1])

  # Write csv double times to make sure the submission is correct,
  # in case some unknown bug when write csv in Colab.
  csv_writer = csv.writer(open('109550058.csv', 'w', newline=''))
  csv_writer.writerow(["id", "failure"])
  csv_writer.writerows(submit)
  csv_writer = csv.writer(open('109550058.csv', 'w', newline=''))
  csv_writer.writerow(["id", "failure"])
  csv_writer.writerows(submit)

In [ ]:
test(x_test, x_test_select)